<a href="https://colab.research.google.com/github/ValentinaZubareva2906/deep_learning_mipt/blob/main/%D0%BD%D0%B5%D0%B4%D0%B5%D0%BB%D1%8F%203%20%22%D0%9E%D1%82%D1%82%D0%BE%D0%BA%20%D0%BA%D0%BB%D0%B8%D0%B5%D0%BD%D1%82%D0%BE%D0%B2%22/note.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>

<h1 style="text-align: center;"><b>Домашнее задание. Решение конкурса на kaggle</b></h1>

<h1 style="text-align: center;"><b>"Предсказание оттока пользователей (весна 2021)"</b></h1>

Цель: моделирование оттока клиентов телеком компании. Метрика на тестовых данных должна быть >= 0.84. Взять метрику ROC-AUC.

Задача:
1) сделать предобаботку данных
2) провести исследовательский анализ
3) построить модели.
4) выбрать наилучшую модель, где на тестовых данных будет score  >= 0.84

Описание данных:
- ClientPeriod -
- MonthlySpending - расходы за месяц
- TotalSpent - общие расходы
- Sex - пол абнимента
- IsSeniorCitizen - является ли абонимент пенсионером
- HasChild - есть ли у клиента дети
- HasPhoneService -
- HasMultiplePhoneNumbers - подключение телефона к нескольким линиям одновременно
- HasInternetService - тип подключения
- HasOnlineSecurityService -  блокировка опасных сайтов
- HasOnlineBackup - облачное хранилище файлов для резервного копирования данных
- HasDeviceProtection - антивирус
- HasTechSupportAccess - выделенная линия технической поддержки
- HasOnlineTV - наличие онлайн телевилдения
- HasMovieSubscription -
- HasContractPhone -
- IsBillingPaperless -
- PaymentMethod - тип платежа


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## Загрузка данных. Предобработка данных.

На данном этапе необходимо:

1) загрузить данные train и test
2) проверить данные на пропуски, дубликаты
3) при обнаружении пропусков необходимо их заменить, при наличии дубликатов, посмотреть стоит ли их удалять.
4) сделать вывод по проделланной предобработке данных.

In [2]:
!gdown 1ERwQ5odiK1Zvi1LtjpkzCMUswYsAX8_K  # train.csv
!gdown 1fGw_-RFwvn_LEdt91Jq-7A-wzG6mmH8r  # test.csv
!gdown 199Mt4OYZNaelT83U-HGDsEYs2YcUGQ6y  # submission.csv

Downloading...
From: https://drive.google.com/uc?id=1ERwQ5odiK1Zvi1LtjpkzCMUswYsAX8_K
To: /content/train.csv
100% 664k/664k [00:00<00:00, 76.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fGw_-RFwvn_LEdt91Jq-7A-wzG6mmH8r
To: /content/test.csv
100% 218k/218k [00:00<00:00, 46.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=199Mt4OYZNaelT83U-HGDsEYs2YcUGQ6y
To: /content/submission.csv
100% 14.7k/14.7k [00:00<00:00, 36.8MB/s]


In [3]:
data_train = pd.read_csv('./train.csv')

In [4]:
display(data_train.head(10))

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
0,55,19.50,1026.35,Male,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,0
1,72,25.85,1872.2,Male,0,Yes,No,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),0
2,1,75.90,75.9,Male,0,No,No,Yes,No,Fiber optic,No,No,No,Yes,No,No,Month-to-month,Yes,Electronic check,1
3,32,79.30,2570,Female,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Mailed check,0
4,60,115.25,6758.45,Female,0,Yes,Yes,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),0
5,25,19.80,475.2,Female,0,No,No,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),0
6,27,90.15,2423.4,Female,0,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,No,Yes,Month-to-month,No,Bank transfer (automatic),0
7,1,45.70,45.7,Male,0,No,No,Yes,No,DSL,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,1
8,50,105.95,5341.8,Male,0,Yes,Yes,Yes,Yes,Fiber optic,Yes,No,Yes,No,Yes,Yes,Month-to-month,No,Credit card (automatic),1
9,72,61.20,4390.25,Male,0,No,No,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),0


In [5]:
print(data_train.columns)

Index(['ClientPeriod', 'MonthlySpending', 'TotalSpent', 'Sex',
       'IsSeniorCitizen', 'HasPartner', 'HasChild', 'HasPhoneService',
       'HasMultiplePhoneNumbers', 'HasInternetService',
       'HasOnlineSecurityService', 'HasOnlineBackup', 'HasDeviceProtection',
       'HasTechSupportAccess', 'HasOnlineTV', 'HasMovieSubscription',
       'HasContractPhone', 'IsBillingPaperless', 'PaymentMethod', 'Churn'],
      dtype='object')


In [6]:
columnname = ['Client_Period', 'Monthly_Spending', 'Total_Spent', 'Sex',
       'Senior_Citizen', 'Partner', 'Child', 'Phone_Service',
       'Multiple_Phone', 'Internet_Service',
       'Online_Service', 'Online_Backup', 'Device_Protection',
       'Tech_Support', 'Online_TV', 'Movie_Subscription',
       'Contract_Phone', 'Billing_Paperless', 'Payment_Method', 'Churn']

data_train = data_train.set_axis(axis=1, labels=columnname)

data_train.columns = data_train.columns.str.lower()

In [7]:
print(data_train.columns)

Index(['client_period', 'monthly_spending', 'total_spent', 'sex',
       'senior_citizen', 'partner', 'child', 'phone_service', 'multiple_phone',
       'internet_service', 'online_service', 'online_backup',
       'device_protection', 'tech_support', 'online_tv', 'movie_subscription',
       'contract_phone', 'billing_paperless', 'payment_method', 'churn'],
      dtype='object')


In [8]:
print(data_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5282 entries, 0 to 5281
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   client_period       5282 non-null   int64  
 1   monthly_spending    5282 non-null   float64
 2   total_spent         5282 non-null   object 
 3   sex                 5282 non-null   object 
 4   senior_citizen      5282 non-null   int64  
 5   partner             5282 non-null   object 
 6   child               5282 non-null   object 
 7   phone_service       5282 non-null   object 
 8   multiple_phone      5282 non-null   object 
 9   internet_service    5282 non-null   object 
 10  online_service      5282 non-null   object 
 11  online_backup       5282 non-null   object 
 12  device_protection   5282 non-null   object 
 13  tech_support        5282 non-null   object 
 14  online_tv           5282 non-null   object 
 15  movie_subscription  5282 non-null   object 
 16  contra

In [9]:
data_test = pd.read_csv('./test.csv')

In [10]:
display(data_test.head(10))

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod
0,42,56.10,2386.85,Male,0,Yes,No,Yes,Yes,DSL,Yes,No,No,No,No,No,One year,No,Credit card (automatic)
1,29,98.50,3004.15,Male,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check
2,9,76.25,684.85,Male,0,No,No,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check
3,67,74.00,4868.4,Female,0,Yes,Yes,Yes,Yes,DSL,Yes,Yes,No,Yes,Yes,No,Two year,No,Credit card (automatic)
4,71,20.10,1389.6,Female,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic)
5,20,25.55,507.4,Female,0,Yes,Yes,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Bank transfer (automatic)
6,9,90.35,767.9,Male,0,Yes,Yes,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,No,One year,Yes,Electronic check
7,41,113.60,4594.95,Female,0,Yes,No,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic)
8,72,109.75,8075.35,Female,1,Yes,No,Yes,Yes,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Two year,Yes,Mailed check
9,23,34.65,768.45,Male,1,No,No,No,No phone service,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Bank transfer (automatic)


In [11]:
print(data_test.columns)

Index(['ClientPeriod', 'MonthlySpending', 'TotalSpent', 'Sex',
       'IsSeniorCitizen', 'HasPartner', 'HasChild', 'HasPhoneService',
       'HasMultiplePhoneNumbers', 'HasInternetService',
       'HasOnlineSecurityService', 'HasOnlineBackup', 'HasDeviceProtection',
       'HasTechSupportAccess', 'HasOnlineTV', 'HasMovieSubscription',
       'HasContractPhone', 'IsBillingPaperless', 'PaymentMethod'],
      dtype='object')


In [12]:
columnname = ['Client_Period', 'Monthly_Spending', 'Total_Spent', 'Sex',
       'Senior_Citizen', 'Partner', 'Child', 'Phone_Service',
       'Multiple_Phone', 'Internet_Service',
       'Online_Service', 'Online_Backup', 'Device_Protection',
       'Tech_Support', 'Online_TV', 'Movie_Subscription',
       'Contract_Phone', 'Billing_Paperless', 'Payment_Method']

data_test = data_test.set_axis(axis=1, labels=columnname)

data_test.columns = data_test.columns.str.lower()

In [13]:
print(data_test.columns)

Index(['client_period', 'monthly_spending', 'total_spent', 'sex',
       'senior_citizen', 'partner', 'child', 'phone_service', 'multiple_phone',
       'internet_service', 'online_service', 'online_backup',
       'device_protection', 'tech_support', 'online_tv', 'movie_subscription',
       'contract_phone', 'billing_paperless', 'payment_method'],
      dtype='object')


In [14]:
print(data_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1761 entries, 0 to 1760
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   client_period       1761 non-null   int64  
 1   monthly_spending    1761 non-null   float64
 2   total_spent         1761 non-null   object 
 3   sex                 1761 non-null   object 
 4   senior_citizen      1761 non-null   int64  
 5   partner             1761 non-null   object 
 6   child               1761 non-null   object 
 7   phone_service       1761 non-null   object 
 8   multiple_phone      1761 non-null   object 
 9   internet_service    1761 non-null   object 
 10  online_service      1761 non-null   object 
 11  online_backup       1761 non-null   object 
 12  device_protection   1761 non-null   object 
 13  tech_support        1761 non-null   object 
 14  online_tv           1761 non-null   object 
 15  movie_subscription  1761 non-null   object 
 16  contra

ВЫВОД: была проведена загрузка данных train и test. Названия колонн в обоих выборках были изменены (приведены к нижнему регистру и сокращены для удобства пользовния).Первичное знакомство показало, что в данных преобладают категоричесские признаки. На этапе предобработки необходимо проверить их на уникальность признаков. Также необходимо посмотреть на дубликаты данных и проверить на пропуски.

In [15]:
print('Количество дубликотов строк в data_train:',data_train.duplicated().sum())
print ('Процент дубликатов строк от всего количества строк:',round(data_train.duplicated().sum() / data_train.shape[0]* 100,3))
print('Количество строк:',data_train.shape[0])

Количество дубликотов строк в data_train: 14
Процент дубликатов строк от всего количества строк: 0.265
Количество строк: 5282


In [16]:
print(data_train.isna().sum())

client_period         0
monthly_spending      0
total_spent           0
sex                   0
senior_citizen        0
partner               0
child                 0
phone_service         0
multiple_phone        0
internet_service      0
online_service        0
online_backup         0
device_protection     0
tech_support          0
online_tv             0
movie_subscription    0
contract_phone        0
billing_paperless     0
payment_method        0
churn                 0
dtype: int64


Проверка на дубликаты строк данных data_train показала, что их составляет 14, что является менее 1%, удалять их не будем, поскольку это могут быть данные разных клиентов.
Проверка на пропуски данных data_train показало, что они отусутствуют.

In [17]:
list = ['partner', 'child', 'phone_service', 'multiple_phone',
       'internet_service', 'online_service', 'online_backup',
       'device_protection', 'tech_support', 'online_tv', 'movie_subscription',
       'contract_phone', 'billing_paperless', 'payment_method']

for word in list:
    print(f'уникальные значения столбца {word}: {data_train[word].unique()}\n')

уникальные значения столбца partner: ['Yes' 'No']

уникальные значения столбца child: ['Yes' 'No']

уникальные значения столбца phone_service: ['Yes' 'No']

уникальные значения столбца multiple_phone: ['No' 'Yes' 'No phone service']

уникальные значения столбца internet_service: ['No' 'Fiber optic' 'DSL']

уникальные значения столбца online_service: ['No internet service' 'No' 'Yes']

уникальные значения столбца online_backup: ['No internet service' 'No' 'Yes']

уникальные значения столбца device_protection: ['No internet service' 'No' 'Yes']

уникальные значения столбца tech_support: ['No internet service' 'Yes' 'No']

уникальные значения столбца online_tv: ['No internet service' 'No' 'Yes']

уникальные значения столбца movie_subscription: ['No internet service' 'No' 'Yes']

уникальные значения столбца contract_phone: ['One year' 'Two year' 'Month-to-month']

уникальные значения столбца billing_paperless: ['No' 'Yes']

уникальные значения столбца payment_method: ['Mailed check' 'Credi

In [18]:
list = ['partner', 'child', 'phone_service', 'multiple_phone',
       'internet_service', 'online_service', 'online_backup',
       'device_protection', 'tech_support', 'online_tv', 'movie_subscription',
       'contract_phone', 'billing_paperless', 'payment_method']

for word in list:
  if word == 'payment_method':
    data_train[word] = data_train[word].replace('Bank transfer (automatic)' ,'Bank transfer').replace('Credit card (automatic)' ,'Credit card')
  data_train[word] = data_train[word].replace('No internet service' ,'No internet')
  data_train[word] = data_train[word].replace('No phone service' ,'No phone')
  data_train[word] = data_train[word].str.replace(' ','_').str.lower()
  print(f'уникальные значения столбца {word}: {data_train[word].unique()}\n')

уникальные значения столбца partner: ['yes' 'no']

уникальные значения столбца child: ['yes' 'no']

уникальные значения столбца phone_service: ['yes' 'no']

уникальные значения столбца multiple_phone: ['no' 'yes' 'no_phone']

уникальные значения столбца internet_service: ['no' 'fiber_optic' 'dsl']

уникальные значения столбца online_service: ['no_internet' 'no' 'yes']

уникальные значения столбца online_backup: ['no_internet' 'no' 'yes']

уникальные значения столбца device_protection: ['no_internet' 'no' 'yes']

уникальные значения столбца tech_support: ['no_internet' 'yes' 'no']

уникальные значения столбца online_tv: ['no_internet' 'no' 'yes']

уникальные значения столбца movie_subscription: ['no_internet' 'no' 'yes']

уникальные значения столбца contract_phone: ['one_year' 'two_year' 'month-to-month']

уникальные значения столбца billing_paperless: ['no' 'yes']

уникальные значения столбца payment_method: ['mailed_check' 'credit_card' 'electronic_check' 'bank_transfer']



In [19]:
print('количество строк, где в категориальных признаках стоит "no_internet":',data_train.query('online_service == "no_internet" and online_backup == "no_internet" and device_protection == "no_internet" and tech_support == "no_internet" and online_tv == "no_internet" and movie_subscription == "no_internet"')['online_service'].count())
print('количество строк, где в признаке online_service стоит "no_internet":', data_train.query('online_service == "no_internet"')['online_service'].count())

количество строк, где в категориальных признаках стоит "no_internet": 1141
количество строк, где в признаке online_service стоит "no_internet": 1141


Проверка на уникальные значения показала, что есть пользователи у которых нет подключения к интернету, их нельзя объединить с пользователями, у которых просто нет какой-то опции.

In [24]:
print('Количество клиентов, у которых общих расходов нет:', data_train.query('total_spent == 0')['total_spent'].count())

Количество клиентов, у которых общих расходов нет: 0


In [25]:
print('Количество клиентов, у которых не указаны общие расходы:', data_train.query('total_spent == " "')['total_spent'].count())

Количество клиентов, у которых не указаны общие расходы: 9


In [26]:
display(data_train.query('total_spent == " "').head(10))

,client_period,monthly_spending,total_spent,sex,senior_citizen,partner,child,phone_service,multiple_phone,internet_service,online_service,online_backup,device_protection,tech_support,online_tv,movie_subscription,contract_phone,billing_paperless,payment_method,churn
1048,0,25.75,,Male,0,yes,yes,yes,yes,no,no_internet,no_internet,no_internet,no_internet,no_internet,no_internet,two_year,no,mailed_check,0
1707,0,73.35,,Female,0,yes,yes,yes,yes,dsl,no,yes,yes,yes,yes,no,two_year,no,mailed_check,0
2543,0,19.70,,Male,0,yes,yes,yes,no,no,no_internet,no_internet,no_internet,no_internet,no_internet,no_internet,one_year,yes,mailed_check,0
3078,0,80.85,,Female,0,yes,yes,yes,no,dsl,yes,yes,yes,no,yes,yes,two_year,no,mailed_check,0
3697,0,20.00,,Female,0,yes,yes,yes,no,no,no_internet,no_internet,no_internet,no_internet,no_internet,no_internet,two_year,no,mailed_check,0
4002,0,61.90,,Male,0,no,yes,yes,yes,dsl,yes,yes,no,yes,no,no,two_year,yes,bank_transfer,0
4326,0,25.35,,Male,0,yes,yes,yes,yes,no,no_internet,no_internet,no_internet,no_internet,no_internet,no_internet,two_year,no,mailed_check,0
4551,0,52.55,,Female,0,yes,yes,no,no_phone,dsl,yes,no,yes,yes,yes,no,two_year,yes,bank_transfer,0
4598,0,56.05,,Female,0,yes,yes,no,no_phone,dsl,yes,yes,yes,yes,yes,no,two_year,no,credit_card,0


In [28]:
data_train['total_spent'] = data_train['total_spent'].str.replace(' ','0').astype('float')

In [30]:
display(data_train[['client_period', 'monthly_spending', 'total_spent']].describe())

,client_period,monthly_spending,total_spent
count,5282.000000,5282.000000,5282.000000
mean,32.397009,64.924754,2287.486161
std,24.550326,30.176464,2269.049195
min,0.000000,18.250000,0.000000
25%,9.000000,35.462500,395.725000
50%,29.000000,70.400000,1405.650000
75%,55.000000,90.050000,3795.212500
max,72.000000,118.750000,8684.800000


Проверка численных признаков данных data_train показала, что в признаке total_spent имеются пропуски, эти пропуски связаны с тем, что клиент только начал пользоваться данной компанией, период пользования составляет 0, следовательно общих его расходов пока нет, поэтому юыло принято поставить заглушку 0. Также был изменен тип данных на float.
У остальных численных признаков нет пропусков. Более детально рассмотрим их на ииследовательском анализе.

In [31]:
print('Количество дубликотов строк в data_test:',data_test.duplicated().sum())
print ('Процент дубликатов строк от всего количества строк:',round(data_test.duplicated().sum() / data_test.shape[0]* 100,3))
print('Количество строк:',data_test.shape[0])

Количество дубликотов строк в data_test: 3
Процент дубликатов строк от всего количества строк: 0.17
Количество строк: 1761


In [32]:
print(data_test.isna().sum())

client_period         0
monthly_spending      0
total_spent           0
sex                   0
senior_citizen        0
partner               0
child                 0
phone_service         0
multiple_phone        0
internet_service      0
online_service        0
online_backup         0
device_protection     0
tech_support          0
online_tv             0
movie_subscription    0
contract_phone        0
billing_paperless     0
payment_method        0
dtype: int64


In [33]:
list = ['partner', 'child', 'phone_service', 'multiple_phone',
       'internet_service', 'online_service', 'online_backup',
       'device_protection', 'tech_support', 'online_tv', 'movie_subscription',
       'contract_phone', 'billing_paperless', 'payment_method']

for word in list:
    print(f'уникальные значения столбца {word}: {data_test[word].unique()}\n')

уникальные значения столбца partner: ['Yes' 'No']

уникальные значения столбца child: ['No' 'Yes']

уникальные значения столбца phone_service: ['Yes' 'No']

уникальные значения столбца multiple_phone: ['Yes' 'No' 'No phone service']

уникальные значения столбца internet_service: ['DSL' 'Fiber optic' 'No']

уникальные значения столбца online_service: ['Yes' 'No' 'No internet service']

уникальные значения столбца online_backup: ['No' 'Yes' 'No internet service']

уникальные значения столбца device_protection: ['No' 'Yes' 'No internet service']

уникальные значения столбца tech_support: ['No' 'Yes' 'No internet service']

уникальные значения столбца online_tv: ['No' 'Yes' 'No internet service']

уникальные значения столбца movie_subscription: ['No' 'Yes' 'No internet service']

уникальные значения столбца contract_phone: ['One year' 'Month-to-month' 'Two year']

уникальные значения столбца billing_paperless: ['No' 'Yes']

уникальные значения столбца payment_method: ['Credit card (automat

In [34]:
list = ['partner', 'child', 'phone_service', 'multiple_phone',
       'internet_service', 'online_service', 'online_backup',
       'device_protection', 'tech_support', 'online_tv', 'movie_subscription',
       'contract_phone', 'billing_paperless', 'payment_method']

for word in list:
  if word == 'payment_method':
    data_test[word] = data_test[word].replace('Bank transfer (automatic)' ,'Bank transfer').replace('Credit card (automatic)' ,'Credit card')
  data_test[word] = data_test[word].replace('No internet service' ,'No internet')
  data_test[word] = data_test[word].replace('No phone service' ,'No phone')
  data_test[word] = data_test[word].str.replace(' ','_').str.lower()
  print(f'уникальные значения столбца {word}: {data_test[word].unique()}\n')

уникальные значения столбца partner: ['yes' 'no']

уникальные значения столбца child: ['no' 'yes']

уникальные значения столбца phone_service: ['yes' 'no']

уникальные значения столбца multiple_phone: ['yes' 'no' 'no_phone']

уникальные значения столбца internet_service: ['dsl' 'fiber_optic' 'no']

уникальные значения столбца online_service: ['yes' 'no' 'no_internet']

уникальные значения столбца online_backup: ['no' 'yes' 'no_internet']

уникальные значения столбца device_protection: ['no' 'yes' 'no_internet']

уникальные значения столбца tech_support: ['no' 'yes' 'no_internet']

уникальные значения столбца online_tv: ['no' 'yes' 'no_internet']

уникальные значения столбца movie_subscription: ['no' 'yes' 'no_internet']

уникальные значения столбца contract_phone: ['one_year' 'month-to-month' 'two_year']

уникальные значения столбца billing_paperless: ['no' 'yes']

уникальные значения столбца payment_method: ['credit_card' 'electronic_check' 'bank_transfer' 'mailed_check']



In [35]:
print('количество строк, где в категориальных признаках стоит "no_internet":',data_test.query('online_service == "no_internet" and online_backup == "no_internet" and device_protection == "no_internet" and tech_support == "no_internet" and online_tv == "no_internet" and movie_subscription == "no_internet"')['online_service'].count())
print('количество строк, где в признаке online_service стоит "no_internet":', data_test.query('online_service == "no_internet"')['online_service'].count())

количество строк, где в категориальных признаках стоит "no_internet": 385
количество строк, где в признаке online_service стоит "no_internet": 385


In [36]:
print('Количество клиентов, у которых общих расходов нет:', data_test.query('total_spent == 0')['total_spent'].count())

Количество клиентов, у которых общих расходов нет: 0


In [37]:
print('Количество клиентов, у которых не указаны общие расходы:', data_test.query('total_spent == " "')['total_spent'].count())

Количество клиентов, у которых не указаны общие расходы: 2


In [38]:
data_test['total_spent'] = data_test['total_spent'].str.replace(' ','0').astype('float')

In [39]:
display(data_test[['client_period', 'monthly_spending', 'total_spent']].describe())

,client_period,monthly_spending,total_spent
count,1761.000000,1761.000000,1761.000000
mean,32.293583,64.272601,2256.483135
std,24.593736,29.832517,2260.501794
min,0.000000,18.550000,0.000000
25%,9.000000,35.650000,405.600000
50%,29.000000,70.050000,1356.300000
75%,56.000000,89.400000,3759.050000
max,72.000000,117.350000,8496.700000


С данными data_test была проведена аналогичная предобработка данных. Было выявлено наличие дубликатов, их составляет менее 1%, было принято решение их не удалять. Было выявлены также пропуски в значениях total_spent, была поставлена заглушка 0.

ВЫВОД: была проведена предобработка данных. Было сделано:
- проведена проверка на наличие явных дубликатов, их составило менее 1% в обоих датасетах, поскольку данных мало, было принято решение из не удалять.
- в категорических признаков была преведена проверка на уникальность значений, было выявлено значение "no_internet". Пользователи у которых нет подключения к интернету, их нельзя объединить с пользователями, у которых просто нет какой-то опции.
- в количественном признаке total_spent были выявлены пропуски, на них была поставлена заглушка 0, поскольку это те клиенты, которые только недавно ими стали.

## Анализ данных (3 балла)

1) Для численных призанков постройте гистограмму (*plt.hist(...)*) или boxplot (*plt.boxplot(...)*). Для категориальных посчитайте количество каждого значения для каждого признака. Для каждой колонки надо сделать *data.value_counts()* и построить bar диаграммы *plt.bar(...)* или круговые диаграммы *plt.pie(...)* (хорошо, елси вы сможете это сделать на одном гарфике с помощью *plt.subplots(...)*).

2) Посмотрите на распределение целевой переменной и скажите, являются ли классы несбалансированными.

3) (Если будет желание) Поиграйте с разными библиотеками для визуализации - *sns*, *pandas_visual_analysis*, etc.

Второй пункт очень важен, потому что существуют задачи классификации с несбалансированными классами. Например, это может значить, что в датасете намного больше примеров 0 класса. В таких случаях нужно 1) не использовать accuracy как метрику 2) использовать методы борьбы с imbalanced dataset (обычно если датасет сильно несбалансирован, т.е. класса 1 в 20 раз меньше класса 0).

In [ ]:
# YOUR CODE

(Дополнительно) Если вы нашли какие-то ошибки в данных или выбросы, то можете их убрать. Тут можно поэксперементировать с обработкой данных как угодно, но не за баллы.

In [ ]:
# YOUR CODE

## Применение линейных моделей (3 балла)

1) Обработайте данные для того, чтобы к ним можно было применить LogisticRegression. Т.е. отнормируйте числовые признаки, а категориальные закодируйте с помощью one-hot-encoding'а.

2) С помощью кроссвалидации или разделения на train/valid выборку протестируйте разные значения гиперпараметра C и выберите лучший (можно тестировать С=100, 10, 1, 0.1, 0.01, 0.001) по метрике ROC-AUC.

Если вы разделяете на train/valid, то используйте LogisticRegressionCV. Он сам при вызове .fit() подберет параметр С. (не забудьте передать scroing='roc_auc', чтобы при кроссвалидации сравнивались значения этой метрики, и refit=True, чтобы при потом модель обучилась на всем датасете с лучшим параметром C).


(более сложный вариант) Если вы будете использовать кроссвалидацию, то преобразования данных и LogisticRegression нужно соединить в один Pipeline с помощью make_pipeline, как это делалось во втором семинаре. Потом pipeline надо передать в GridSearchCV. Для one-hot-encoding'a можно испльзовать комбинацию LabelEncoder + OneHotEncoder (сначала превращаем строчки в числа, а потом числа првращаем в one-hot вектора.)

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline

In [ ]:
# YOUR CODE

Выпишите какое лучшее качество и с какими параметрами вам удалось получить

## Применение градиентного бустинга (2 балла)

Если вы хотите получить баллы за точный ответ, то стоит попробовать градиентный бустинг. Часто градиентный бустинг с параметрами по умолчанию даст вам 80% результата за 0% усилий.

Мы будем использовать catboost, поэтому нам не надо кодировать категориальные признаки. Catboost сделает это сам (в .fit() надо передать cat_features=cat_cols). А численные признаки нормировать для моделей, основанных на деревьях не нужно.

1) Разделите выборку на train/valid. Протестируйте catboost cо стандартными параметрами.

2) Протестируйте разные занчения параметров количества деревьев и learning_rate'а и выберите лучшую по метрике ROC-AUC комбинацию.

(Дополнительно) Есть некоторые сложности с тем, чтобы использовать CatBoostClassifier вместе с GridSearchCV, поэтому мы не просим использовать кроссвалидацию. Но можете попробовать)

In [ ]:
# YOUR CODE

Выпишите, какое лучшее качество и с какими параметрами вам удалось получить

ВАШ ОТВЕТ

# Предсказания

In [ ]:
best_model = # какая-то предыдущая модель

In [ ]:
X_test = pd.read_csv('./test.csv')
submission = pd.read_csv('./submission.csv')

submission['Churn'] = # используйте best_model.predict_proba(X_test), не забудьте выделить вероятность класса 1.
submission.to_csv('./my_submission.csv')

Лучшее решение отправьте в Stepik.